In [1]:
import altair as alt
from altair.expr import datum, if_
import datetime
from functools import reduce
import json
import numpy as np
import pandas as pd
import psycopg2
pd.options.display.float_format = "{:,.2f}".format

Connect to the database.

In [2]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf["database"],
    user=conf["user"],
    host=conf["host"],
    password=conf["password"]
)
conn.autocommit = True

## Return the donors who gave at least $50,000 to conservative and GOP-aligned groups between Nov. 2 and Dec. 31, 2017, split by their giving before the tax bill's introduction on Nov. 2 and after its introduction. Do this for every off-year going back to the 2010 cycle.

In [3]:
def run_donors_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """+prefix+"""_donors_committees_"""+suffix+""" AS
                    SELECT match_id,
                           organizations,
                           contributors,
                           sum(committee_total) AS total,
                           cmte_id,
                           pacshort AS committee
                    FROM
                      (SELECT match_id,
                              organizations,
                              contributors,
                              sum(amount) AS committee_total,
                              cmteid AS cmte_id
                       FROM
                         (SELECT CASE
                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                     ELSE orgname
                                 END AS match_id,
                                 array_agg(DISTINCT orgname) AS organizations,
                                 array_agg(DISTINCT contrib) AS contributors
                          FROM crp_contributions
                          LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
                          AND crp_committees.cycle = '""" + reference_cycle + """'
                          WHERE primcode IN ('J1100',
                                             'J2200',
                                             'J2400',
                                             'Z1100',
                                             'Z4100',
                                             'Z4500',
                                             'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
                            AND TYPE NOT IN ('11J',
                                             '15C',
                                             '15E',
                                             '15I',
                                             '15J',
                                             '15T',
                                             '18J',
                                             '19J',
                                             '30J',
                                             '30F',
                                             '31J',
                                             '31F',
                                             '32J',
                                             '32F')
                          GROUP BY match_id
                          HAVING sum(amount) >= 50000) AS gop_donors
                       JOIN crp_contributions ON CASE
                                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                                     ELSE crp_contributions.orgname
                                                 END = gop_donors.match_id
                       WHERE date >= '""" + start + """'
                         AND date <= '""" + end + """'
                         AND CYCLE = '""" + cycle + """'
                         AND TYPE NOT IN ('11J',
                                          '15C',
                                          '15E',
                                          '15I',
                                          '15J',
                                          '15T',
                                          '18J',
                                          '19J',
                                          '30J',
                                          '30F',
                                          '31J',
                                          '31F',
                                          '32J',
                                          '32F')
                       GROUP BY match_id,
                                organizations,
                                contributors,
                                cmte_id) AS donors_committees
                    JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
                    WHERE primcode IN ('J1100',
                                       'J2200',
                                       'J2400',
                                       'Z1100',
                                       'Z4100',
                                       'Z4500',
                                       'Z5100')
                    GROUP BY match_id,
                             organizations,
                             contributors,
                             cmte_id,
                             committee;

                    SELECT *
                    FROM """+prefix+"""_donors_committees_"""+suffix+""";
                    """
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [4]:
post_bill_donors_committees_17 = run_donors_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_donors_committees_17 = run_donors_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

In [5]:
post_bill_donors_committees_15 = run_donors_query("post_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-11-02", "2015-12-31")
pre_bill_donors_committees_15 = run_donors_query("pre_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-01-01", "2015-11-01")

In [6]:
post_bill_donors_committees_13 = run_donors_query("post_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-11-02", "2013-12-31")
pre_bill_donors_committees_13 = run_donors_query("pre_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-01-01", "2013-11-01")

In [7]:
post_bill_donors_committees_11 = run_donors_query("post_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-11-02", "2011-12-31")
pre_bill_donors_committees_11 = run_donors_query("pre_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-01-01", "2011-11-01")

In [8]:
post_bill_donors_committees_09 = run_donors_query("post_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-11-02", "2009-12-31")
pre_bill_donors_committees_09 = run_donors_query("pre_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-01-01", "2009-11-01")

## How much did each contributor give in each period for each cycle?

### 2017

In [9]:
post_bill_donors_17 = post_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_17 = pre_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [10]:
donors_17 = post_bill_donors_17.merge(pre_bill_donors_17, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_17.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_17.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_17["total_pre_bill"].fillna(0, inplace=True)
donors_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 5 columns):
match_id           158 non-null object
organizations      158 non-null object
contributors       158 non-null object
total_post_bill    158 non-null float64
total_pre_bill     158 non-null float64
dtypes: float64(2), object(3)
memory usage: 7.4+ KB


In [11]:
donors_17["pct_post_bill"] = donors_17["total_post_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["pct_pre_bill"] = donors_17["total_pre_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["change"] = donors_17["total_post_bill"] - donors_17["total_pre_bill"]
donors_17["pct_change"] = (donors_17["total_post_bill"] - donors_17["total_pre_bill"]) / donors_17["total_pre_bill"].abs()
donors_17.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
52,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","4,557,300.00","12,051,400.00",0.27,0.73,"-7,494,100.00",-0.62
2,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],"3,747,520.00","9,142,525.00",0.29,0.71,"-5,395,005.00",-0.59
60,U0000004054,"[Ghpalmer Assoc, GH Palmer Assoc]","[PALMER, GEOFF, PALMER, GEOFFREY H]","1,955,200.00","546,272.00",0.78,0.22,"1,408,928.00",2.58
10,Hillwood Development,[Hillwood Development],[HILLWOOD DEVELOPMENT COMPANY LLC],"1,500,000.00","500,000.00",0.75,0.25,"1,000,000.00",2.00
67,U0000004552,"[Cinemark Holdings, Cinemark USA]","[MITCHELL, LEE, MITCHELL, LEE ROY, MITCHELL, T...","1,007,400.00","131,299.00",0.88,0.12,"876,101.00",6.67


#### How much did they give?

In [12]:
print("2017 post-bill total: $" + str(donors_17["total_post_bill"].sum()))
print("2017 pre-bill total: $" + str(donors_17["total_pre_bill"].sum()))

2017 post-bill total: $35639563.0
2017 pre-bill total: $58975282.0


### 2015

In [13]:
post_bill_donors_15 = post_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_15 = pre_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [14]:
donors_15 = post_bill_donors_15.merge(pre_bill_donors_15, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_15.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_15.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_15["total_pre_bill"].fillna(0, inplace=True)
donors_15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 0 to 110
Data columns (total 5 columns):
match_id           111 non-null object
organizations      80 non-null object
contributors       80 non-null object
total_post_bill    80 non-null float64
total_pre_bill     111 non-null float64
dtypes: float64(2), object(3)
memory usage: 5.2+ KB


In [15]:
donors_15["pct_post_bill"] = donors_15["total_post_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["pct_pre_bill"] = donors_15["total_pre_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["change"] = donors_15["total_post_bill"] - donors_15["total_pre_bill"]
donors_15["pct_change"] = (donors_15["total_post_bill"] - donors_15["total_pre_bill"]) / donors_15["total_pre_bill"].abs()
donors_15.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
21,U0000003477,[Koch Industries],"[KOCH, CHARLES, KOCH, CHARLES G MR, KOCH, CHAR...","3,000,000.00",0.00,1.00,0.00,"3,000,000.00",inf
31,U0000003829,[Point72 Asset Management],"[COHEN, STEVE MR]","2,000,000.00","2,002,699.00",0.50,0.50,"-2,699.00",-0.00
4,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],"750,000.00","3,450,000.00",0.18,0.82,"-2,700,000.00",-0.78
27,U0000003682,[Renaissance Technologies],"[MERCER, DIANA, MERCER, DIANA MRS, MERCER, ROB...","560,500.00","14,517,100.00",0.04,0.96,"-13,956,600.00",-0.96
8,U0000000175,[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]","549,400.00","3,485,123.00",0.14,0.86,"-2,935,723.00",-0.84


#### How much did they give?

In [16]:
print("2015 post-bill total: $" + str(donors_15["total_post_bill"].sum()))
print("2015 pre-bill total: $" + str(donors_15["total_pre_bill"].sum()))

2015 post-bill total: $1647560.0
2015 pre-bill total: $77617350.0


### 2013

In [17]:
post_bill_donors_13 = post_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_13 = pre_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [18]:
donors_13 = post_bill_donors_13.merge(pre_bill_donors_13, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_13.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_13.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_13["total_pre_bill"].fillna(0, inplace=True)
donors_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 0 to 108
Data columns (total 5 columns):
match_id           109 non-null object
organizations      62 non-null object
contributors       62 non-null object
total_post_bill    62 non-null float64
total_pre_bill     109 non-null float64
dtypes: float64(2), object(3)
memory usage: 5.1+ KB


In [19]:
donors_13["pct_post_bill"] = donors_13["total_post_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["pct_pre_bill"] = donors_13["total_pre_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["change"] = donors_13["total_post_bill"] - donors_13["total_pre_bill"]
donors_13["pct_change"] = (donors_13["total_post_bill"] - donors_13["total_pre_bill"]) / donors_13["total_pre_bill"].abs()
donors_13.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
3,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],"1,000,000.00",0.00,1.00,0.00,"1,000,000.00",inf
25,U0000003903,[Alliance Coal],"[CRAFT, JOE W III, CRAFT, JOSEPH MR III, CRAFT...","515,600.00","162,900.00",0.76,0.24,"352,700.00",2.17
7,U0000000175,[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]","442,400.00","51,300.00",0.90,0.10,"391,100.00",7.62
22,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","261,400.00","533,150.00",0.33,0.67,"-271,750.00",-0.51
0,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],"109,714.00","81,237.00",0.57,0.43,"28,477.00",0.35


#### How much did they give?

In [20]:
print("2013 post-bill total: $" + str(donors_13["total_post_bill"].sum()))
print("2013 pre-bill total: $" + str(donors_13["total_pre_bill"].sum()))

2013 post-bill total: $3859464.0
2013 pre-bill total: $6716175.0


### 2011

In [21]:
post_bill_donors_11 = post_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_11 = pre_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [22]:
donors_11 = post_bill_donors_11.merge(pre_bill_donors_11, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_11.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_11.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_11["total_pre_bill"].fillna(0, inplace=True)
donors_11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 105
Data columns (total 5 columns):
match_id           106 non-null object
organizations      71 non-null object
contributors       71 non-null object
total_post_bill    71 non-null float64
total_pre_bill     106 non-null float64
dtypes: float64(2), object(3)
memory usage: 5.0+ KB


In [23]:
donors_11["pct_post_bill"] = donors_11["total_post_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["pct_pre_bill"] = donors_11["total_pre_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["change"] = donors_11["total_post_bill"] - donors_11["total_pre_bill"]
donors_11["pct_change"] = (donors_11["total_post_bill"] - donors_11["total_pre_bill"]) / donors_11["total_pre_bill"].abs()
donors_11.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
27,U0000004022,[Thiel Capital],"[THIEL, PETER]","1,035,000.00","32,200.00",0.97,0.03,"1,002,800.00",31.14
11,U0000003260,"[Homemaker, Mt Vernon Investments, Winstar Farm]","[TROUTT, KENNETH A, TROUTT, KENNY A MR, TROUTT...","540,800.00","312,400.00",0.63,0.37,"228,400.00",0.73
22,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","385,200.00","188,300.00",0.67,0.33,"196,900.00",1.05
33,U0000004393,"[Nevada Restaurant Services, Pere Antoine]","[ESTEY, ALLYSON, ESTEY, PATRICIA RIVERS MS]","102,400.00",0.00,1.00,0.00,"102,400.00",inf
2,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],"95,000.00","73,605.00",0.56,0.44,"21,395.00",0.29


#### How much did they give?

In [24]:
print("2011 post-bill total: $" + str(donors_11["total_post_bill"].sum()))
print("2011 pre-bill total: $" + str(donors_11["total_pre_bill"].sum()))

2011 post-bill total: $3241839.0
2011 pre-bill total: $7409350.0


### 2009

In [25]:
post_bill_donors_09 = post_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_09 = pre_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [26]:
donors_09 = post_bill_donors_09.merge(pre_bill_donors_09, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_09.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_09.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_09["total_pre_bill"].fillna(0, inplace=True)
donors_09.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 5 columns):
match_id           88 non-null object
organizations      50 non-null object
contributors       50 non-null object
total_post_bill    50 non-null float64
total_pre_bill     88 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


In [27]:
donors_09["pct_post_bill"] = donors_09["total_post_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["pct_pre_bill"] = donors_09["total_pre_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["change"] = donors_09["total_post_bill"] - donors_09["total_pre_bill"]
donors_09["pct_change"] = (donors_09["total_post_bill"] - donors_09["total_pre_bill"]) / donors_09["total_pre_bill"].abs()
donors_09.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
0,Chickasaw Nation,[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...","75,600.00","96,700.00",0.44,0.56,"-21,100.00",-0.22
45,i3003697615,"[Mpi Research, Northwood Group]","[PARFET, BARBARA A MRS, PARFET, WILLIAM U, PAR...","51,400.00","1,000.00",0.98,0.02,"50,400.00",50.40
38,h1001217591,[Haworth Inc],"[HAWORTH, ETHELYN, HAWORTH, ETHELYN MRS, HAWOR...","40,000.00","2,400.00",0.94,0.06,"37,600.00",15.67
19,U0000003951,[Intercontinentalexchange Inc],"[LOEFFLER, KELLY L MS, SPRECHER, JEFFREY CRAIG...","37,400.00","19,910.00",0.65,0.35,"17,490.00",0.88
14,U0000003654,[Schweitzer Engineering Laboratories],"[SCHWEITZER, BEATRIZ, SCHWEITZER, EDMUND O III]","36,500.00","4,200.00",0.90,0.10,"32,300.00",7.69


#### How much did they give?

In [28]:
print("2009 post-bill total: $" + str(donors_09["total_post_bill"].sum()))
print("2009 pre-bill total: $" + str(donors_09["total_pre_bill"].sum()))

2009 post-bill total: $671846.0
2009 pre-bill total: $1949651.0


## How did each contributor's giving pattern change in each cycle compared with 2017?

### 2015

In [29]:
donors_15_17 = donors_17.merge(donors_15, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17.drop(["change_17", "pct_change_17", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_15_17 = donors_15_17[["match_id", "contributors", "organizations", "total_pre_bill_15", "total_post_bill_15",
                 "pct_pre_bill_15", "pct_post_bill_15", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_15_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_15     111 non-null float64
total_post_bill_15    80 non-null float64
pct_pre_bill_15       80 non-null float64
pct_post_bill_15      80 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [30]:
donors_15_17["giving_change"] = np.where((donors_15_17["pct_post_bill_17"] > donors_15_17["pct_post_bill_15"]) | (donors_15_17["pct_pre_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17["pct_post_bill_17"] < donors_15_17["pct_post_bill_15"], "Decreased",
                                           np.where(donors_15_17["pct_post_bill_17"] == donors_15_17["pct_post_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17.head(1)

,match_id,contributors,organizations,total_pre_bill_15,total_post_bill_15,pct_pre_bill_15,pct_post_bill_15,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


### 2013

In [31]:
donors_13_17 = donors_17.merge(donors_13, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17.drop(["change_17", "pct_change_17", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_13_17 = donors_13_17[["match_id", "contributors", "organizations", "total_pre_bill_13", "total_post_bill_13",
                 "pct_pre_bill_13", "pct_post_bill_13", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_13_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_13     109 non-null float64
total_post_bill_13    62 non-null float64
pct_pre_bill_13       62 non-null float64
pct_post_bill_13      62 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [32]:
donors_13_17["giving_change"] = np.where((donors_13_17["pct_post_bill_17"] > donors_13_17["pct_post_bill_13"]) | (donors_13_17["pct_pre_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17["pct_post_bill_17"] < donors_13_17["pct_post_bill_13"], "Decreased",
                                           np.where(donors_13_17["pct_post_bill_17"] == donors_13_17["pct_post_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17.head(1)

,match_id,contributors,organizations,total_pre_bill_13,total_post_bill_13,pct_pre_bill_13,pct_post_bill_13,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


### 2011

In [33]:
donors_11_17 = donors_17.merge(donors_11, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17.drop(["change_17", "pct_change_17", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_11_17 = donors_11_17[["match_id", "contributors", "organizations", "total_pre_bill_11", "total_post_bill_11",
                 "pct_pre_bill_11", "pct_post_bill_11", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_11_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_11     106 non-null float64
total_post_bill_11    71 non-null float64
pct_pre_bill_11       71 non-null float64
pct_post_bill_11      71 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [34]:
donors_11_17["giving_change"] = np.where((donors_11_17["pct_post_bill_17"] > donors_11_17["pct_post_bill_11"]) | (donors_11_17["pct_pre_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17["pct_post_bill_17"] < donors_11_17["pct_post_bill_11"], "Decreased",
                                           np.where(donors_11_17["pct_post_bill_17"] == donors_11_17["pct_post_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17.head(1)

,match_id,contributors,organizations,total_pre_bill_11,total_post_bill_11,pct_pre_bill_11,pct_post_bill_11,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],"2,500.00",nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


### 2009

In [35]:
donors_09_17 = donors_17.merge(donors_09, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17.drop(["change_17", "pct_change_17", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_09_17 = donors_09_17[["match_id", "contributors", "organizations", "total_pre_bill_09", "total_post_bill_09",
                 "pct_pre_bill_09", "pct_post_bill_09", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_09_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_09     88 non-null float64
total_post_bill_09    50 non-null float64
pct_pre_bill_09       50 non-null float64
pct_post_bill_09      50 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [36]:
donors_09_17["giving_change"] = np.where((donors_09_17["pct_post_bill_17"] > donors_09_17["pct_post_bill_09"]) | (donors_09_17["pct_pre_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17["pct_post_bill_17"] < donors_09_17["pct_post_bill_09"], "Decreased",
                                           np.where(donors_09_17["pct_post_bill_17"] == donors_09_17["pct_post_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17.head(1)

,match_id,contributors,organizations,total_pre_bill_09,total_post_bill_09,pct_pre_bill_09,pct_post_bill_09,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


## How many contributors gave more money in 2017 than in any prior cycle?

In [37]:
donors_all_years_list = [donors_09, donors_11, donors_13, donors_15, donors_17]

In [38]:
donors_all_years = reduce(lambda left, right: pd.merge(left, right, on="match_id", how="outer"), donors_all_years_list)

In [39]:
donors_all_years.drop(["organizations_x", "organizations_y", "contributors_x", "contributors_y", "change_x", "change_y", "change", "pct_change_x", "pct_change_y",
                       "pct_change"], axis=1, inplace=True)
donors_all_years.columns = ["match_id", "total_post_bill_09", "total_pre_bill_09", "pct_post_bill_09", "pct_pre_bill_09", "total_post_bill_11",
                            "total_pre_bill_11", "pct_post_bill_11", "pct_pre_bill_11", "total_post_bill_13", "total_pre_bill_13", "pct_post_bill_13",
                            "pct_pre_bill_13", "total_post_bill_15", "total_pre_bill_15", "pct_post_bill_15", "pct_pre_bill_15", "organizations", "contributors",
                            "total_post_bill_17", "total_pre_bill_17", "pct_post_bill_17", "pct_pre_bill_17"]
donors_all_years = donors_all_years[["match_id", "organizations", "contributors", "total_post_bill_09", "total_pre_bill_09", "pct_post_bill_09", "pct_pre_bill_09", "total_post_bill_11",
                            "total_pre_bill_11", "pct_post_bill_11", "pct_pre_bill_11", "total_post_bill_13", "total_pre_bill_13", "pct_post_bill_13",
                            "pct_pre_bill_13", "total_post_bill_15", "total_pre_bill_15", "pct_post_bill_15", "pct_pre_bill_15", "total_post_bill_17",
                                    "total_pre_bill_17", "pct_post_bill_17", "pct_pre_bill_17"]]
donors_all_years.fillna(0, inplace=True)
donors_all_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 23 columns):
match_id              158 non-null object
organizations         158 non-null object
contributors          158 non-null object
total_post_bill_09    158 non-null float64
total_pre_bill_09     158 non-null float64
pct_post_bill_09      158 non-null float64
pct_pre_bill_09       158 non-null float64
total_post_bill_11    158 non-null float64
total_pre_bill_11     158 non-null float64
pct_post_bill_11      158 non-null float64
pct_pre_bill_11       158 non-null float64
total_post_bill_13    158 non-null float64
total_pre_bill_13     158 non-null float64
pct_post_bill_13      158 non-null float64
pct_pre_bill_13       158 non-null float64
total_post_bill_15    158 non-null float64
total_pre_bill_15     158 non-null float64
pct_post_bill_15      158 non-null float64
pct_pre_bill_15       158 non-null float64
total_post_bill_17    158 non-null float64
total_pre_bill_17     158 non-null flo

In [40]:
print(str(donors_all_years[(donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_15"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_13"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_11"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_09"])]["match_id"].count()) + " donors increased the share of their annual giving that fell in the last two months of the year in 2017 as compared with every prior odd-numbered year.")
print("Among those donors, " + (str(donors_all_years[(donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_15"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_13"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_11"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_09"]) &
                                                     (donors_all_years["pct_post_bill_15"] != 0) &
                                                     (donors_all_years["pct_post_bill_13"] != 0) &
                                                     (donors_all_years["pct_post_bill_11"] != 0) &
                                                     (donors_all_years["pct_post_bill_09"] != 0)]["match_id"].count())) + " gave some amount of money in every prior odd-numbered year")

95 donors increased the share of their annual giving that fell in the last two months of the year in 2017 as compared with every prior odd-numbered year.
Among those donors, 8 gave some amount of money in every prior odd-numbered year


## And how many contributors gave more in 2017 than in all but one prior cycle?

In [41]:
print(str(donors_all_years[((donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_15"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_13"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_11"])) |
                ((donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_15"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_13"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_09"])) |
                ((donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_15"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_11"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_09"])) |
                ((donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_13"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_11"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_09"]))]["match_id"].count()) + " donors increased the share of their annual giving that fell in the last two months of the year in 2017 as compared with all but one prior odd-numbered year.")

139 donors increased the share of their annual giving that fell in the last two months of the year in 2017 as compared with all but one prior odd-numbered year.


## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in each of the previous years?

In [42]:
print(str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["match_id"].count() / donors_15_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2015 and 2017.")
print(str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["match_id"].count() / donors_13_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2013 and 2017.")
print(str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["match_id"].count() / donors_11_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2011 and 2017.")
print(str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["match_id"].count() / donors_09_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2009 and 2017.")

89.24050632911393% of donors increased the share of their annual giving that fell in the last two months of the year between 2015 and 2017.
83.54430379746836% of donors increased the share of their annual giving that fell in the last two months of the year between 2013 and 2017.
85.44303797468355% of donors increased the share of their annual giving that fell in the last two months of the year between 2011 and 2017.
85.44303797468355% of donors increased the share of their annual giving that fell in the last two months of the year between 2009 and 2017.


## And by how much did these individuals' giving increase?

In [43]:
print("These donors' giving in the last two months of the year increased by $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()) + ", from $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()) + " to $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2015 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()) + ", from $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()) + " to $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2013 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()) + ", from $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()) + " to $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2011 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()) + ", from $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()) + " to $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2009 and 2017.")

These donors' giving in the last two months of the year increased by $35512029.0, from $-6589136.0 to $28922893.0 between 2015 and 2017.
These donors' giving in the last two months of the year increased by $21880518.0, from $1155975.0 to $23036493.0 between 2013 and 2017.
These donors' giving in the last two months of the year increased by $24340851.0, from $613592.0 to $24954443.0 between 2011 and 2017.
These donors' giving in the last two months of the year increased by $31881913.0, from $267350.0 to $32149263.0 between 2009 and 2017.


## How many donors in each year gave all of their money in the last two months of each year?

In [44]:
print("In 2017, " + str(donors_17[donors_17["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2015, " + str(donors_15[donors_15["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2013, " + str(donors_13[donors_13["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2011, " + str(donors_11[donors_11["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2009, " + str(donors_09[donors_09["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")

In 2017, 44 donors gave all their money in the last two months.
In 2015, 2 donors gave all their money in the last two months.
In 2013, 4 donors gave all their money in the last two months.
In 2011, 3 donors gave all their money in the last two months.
In 2009, 7 donors gave all their money in the last two months.


# What does this giving look like on a contribution-by-contribution level?

In [45]:
def run_contributions_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_contributions_""" + suffix + """ AS
    SELECT fectransid,
    match_id,
    organization,
    contributor, date, sum(SUM),
    cmte_id,
    pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
                       WHERE date >= '""" + start + """'
                         AND date <= '""" + end + """'
                         AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;
                            SELECT *
                            FROM """ + prefix + """_contributions_"""+ suffix +""";"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [46]:
post_bill_contributions_17 = run_contributions_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_contributions_17 = run_contributions_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

In [47]:
post_bill_contributions_15 = run_contributions_query("post_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-11-02", "2015-12-31")
pre_bill_contributions_15 = run_contributions_query("pre_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-01-01", "2015-11-01")

In [48]:
post_bill_contributions_13 = run_contributions_query("post_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-11-02", "2013-12-31")
pre_bill_contributions_13 = run_contributions_query("pre_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-01-01", "2013-11-01")

In [49]:
post_bill_contributions_11 = run_contributions_query("post_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-11-02", "2011-12-31")
pre_bill_contributions_11 = run_contributions_query("pre_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-01-01", "2011-11-01")

In [50]:
post_bill_contributions_09 = run_contributions_query("post_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-11-02", "2009-12-31")
pre_bill_contributions_09 = run_contributions_query("pre_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-01-01", "2009-11-01")

## How many of these donors gave to Democrats in 2017?

In [51]:
def run_democratic_donors_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """+prefix+"""_democratic_donors_committees_"""+suffix+""" AS
                    SELECT match_id,
                           organizations,
                           contributors,
                           sum(committee_total) AS total,
                           cmte_id,
                           pacshort AS committee
                    FROM
                      (SELECT match_id,
                              organizations,
                              contributors,
                              sum(amount) AS committee_total,
                              cmteid AS cmte_id
                       FROM
                         (SELECT CASE
                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                     ELSE orgname
                                 END AS match_id,
                                 array_agg(DISTINCT orgname) AS organizations,
                                 array_agg(DISTINCT contrib) AS contributors
                          FROM crp_contributions
                          LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
                          AND crp_committees.cycle = '""" + reference_cycle + """'
                          WHERE primcode IN ('J1100',
                                             'J2200',
                                             'J2400',
                                             'Z1100',
                                             'Z4100',
                                             'Z4500',
                                             'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
                            AND TYPE NOT IN ('11J',
                                             '15C',
                                             '15E',
                                             '15I',
                                             '15J',
                                             '15T',
                                             '18J',
                                             '19J',
                                             '30J',
                                             '30F',
                                             '31J',
                                             '31F',
                                             '32J',
                                             '32F')
                          GROUP BY match_id
                          HAVING sum(amount) >= 50000) AS gop_donors
                       JOIN crp_contributions ON CASE
                                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                                     ELSE crp_contributions.orgname
                                                 END = gop_donors.match_id
                       WHERE date >= '""" + start + """'
                         AND date <= '""" + end + """'
                         AND CYCLE = '""" + cycle + """'
                         AND TYPE NOT IN ('11J',
                                          '15C',
                                          '15E',
                                          '15I',
                                          '15J',
                                          '15T',
                                          '18J',
                                          '19J',
                                          '30J',
                                          '30F',
                                          '31J',
                                          '31F',
                                          '32J',
                                          '32F')
                       GROUP BY match_id,
                                organizations,
                                contributors,
                                cmte_id) AS donors_committees
                    JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
                    WHERE primcode IN ('J1200',
                                       'J2100',
                                       'J2300',
                                       'Z1200',
                                       'Z4200',
                                       'Z5200',
                                       'Z4400')
                    GROUP BY match_id,
                             organizations,
                             contributors,
                             cmte_id,
                             committee;

                    SELECT *
                    FROM """+prefix+"""_democratic_donors_committees_"""+suffix+""";
                    """
    return pd.read_sql(query, con=conn)

## Return the data for 2017.

In [52]:
post_bill_democratic_donors_committees_17 = run_democratic_donors_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_democratic_donors_committees_17 = run_democratic_donors_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

## How much did each contributor give in each period?

In [53]:
post_bill_democratic_donors_17 = post_bill_democratic_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_democratic_donors_17 = pre_bill_democratic_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [54]:
democratic_donors_17 = post_bill_democratic_donors_17.merge(pre_bill_democratic_donors_17, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
democratic_donors_17["total_pre_bill"].fillna(0, inplace=True)
democratic_donors_17["total_post_bill"].fillna(0, inplace=True)
democratic_donors_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 7 columns):
match_id                   31 non-null object
organizations_post_bill    14 non-null object
contributors_post_bill     14 non-null object
total_post_bill            31 non-null float64
organizations_pre_bill     31 non-null object
contributors_pre_bill      31 non-null object
total_pre_bill             31 non-null float64
dtypes: float64(2), object(5)
memory usage: 1.9+ KB


In [55]:
democratic_donors_17["pct_post_bill"] = democratic_donors_17["total_post_bill"] / (democratic_donors_17["total_pre_bill"] + democratic_donors_17["total_post_bill"])
democratic_donors_17["pct_pre_bill"] = democratic_donors_17["total_pre_bill"] / (democratic_donors_17["total_pre_bill"] + democratic_donors_17["total_post_bill"])
democratic_donors_17["change"] = democratic_donors_17["total_post_bill"] - democratic_donors_17["total_pre_bill"]
democratic_donors_17["pct_change"] = (democratic_donors_17["total_post_bill"] - democratic_donors_17["total_pre_bill"]) / democratic_donors_17["total_pre_bill"].abs()
democratic_donors_17.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations_post_bill,contributors_post_bill,total_post_bill,organizations_pre_bill,contributors_pre_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
1,Entertainment Software Assn,[Entertainment Software Assn],[ENTERTAINMENT SOFTWARE ASSOCIATION],"100,000.00",[Entertainment Software Assn],[ENTERTAINMENT SOFTWARE ASSOCIATION],"50,000.00",0.67,0.33,"50,000.00",1.00
2,Shakopee Mdewakanton Sioux Community,[Shakopee Mdewakanton Sioux Community],"[MDEWAKANTON SIOUX COMMUNITY, SHAKOPEE,,,, SHA...","61,700.00",[Shakopee Mdewakanton Sioux Community],"[MDEWAKANTON SIOUX COMMUNITY, SHAKOPEE,,,, SHA...","93,700.00",0.40,0.60,"-32,000.00",-0.34
4,U0000000386,[Avenue Ventures],"[ZUBERI, IMAAD, ZUBERI, IMAAD MR]","47,100.00",[Avenue Ventures],"[ZUBERI, IMAAD, ZUBERI, IMAAD MR]","65,456.00",0.42,0.58,"-18,356.00",-0.28
10,e1110767126,[Holland Partners],"[HOLLAND, CLYDE, HOLLAND, CLYDE P, HOLLAND, CL...","25,000.00",[Holland Partners],"[HOLLAND, CLYDE, HOLLAND, CLYDE P, HOLLAND, CL...","5,400.00",0.82,0.18,"19,600.00",3.63
7,a0000027527,"[Alcalde & Fay, Camival Corp, Carnival Corp, C...","[ARISON, MADELEINE, ARISON, MADELEINE MRS, ARI...","25,000.00","[Alcalde & Fay, Camival Corp, Carnival Corp, C...","[ARISON, MADELEINE, ARISON, MADELEINE MRS, ARI...","40,600.00",0.38,0.62,"-15,600.00",-0.38


In [56]:
democratic_donors_17["match_id"].count()

31

#### How much did they give?

In [57]:
print(str(democratic_donors_17["match_id"].count()) + " donors gave a combined $" + str(democratic_donors_17["total_post_bill"].sum() +
                                                                                        democratic_donors_17["total_pre_bill"].sum()) +
      " to liberal-leaning and Democratic groups over the course of 2017. Of that amount, $" +
      str(democratic_donors_17["total_pre_bill"].sum()) + " came before the bill was introduced and $" +
      str(democratic_donors_17["total_post_bill"].sum()) + " came after the bill was introduced.")

31 donors gave a combined $1196591.0 to liberal-leaning and Democratic groups over the course of 2017. Of that amount, $877091.0 came before the bill was introduced and $319500.0 came after the bill was introduced.


# What does this giving look like on a contribution-by-contribution level?

In [58]:
def run_democratic_contributions_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_democratic_contributions_""" + suffix + """ AS
    SELECT fectransid,
    match_id,
    organization,
    contributor, date, sum(SUM),
    cmte_id,
    pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
                       WHERE date >= '""" + start + """'
                         AND date <= '""" + end + """'
                         AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
                    WHERE primcode IN ('J1200',
                                       'J2100',
                                       'J2300',
                                       'Z1200',
                                       'Z4200',
                                       'Z5200',
                                       'Z4400')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;
                            SELECT *
                            FROM """ + prefix + """_democratic_contributions_"""+ suffix +""";"""
    return pd.read_sql(query, con=conn)

## Return the data for 2017.

In [59]:
post_bill_democratic_contributions_17 = run_democratic_contributions_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_democratic_contributions_17 = run_democratic_contributions_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

## Return the donors who gave at least $50,000 to conservative and GOP-aligned groups between Feb. 2 and March 31, 2017, split by their giving inside and outside of that period. Do this for every off-year going back to the 2010 cycle.

In [60]:
def run_donors_query_1st_qtr(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start_one, end_one, start_two, end_two):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """+prefix+"""_donors_committees_"""+suffix+""" AS
                    SELECT match_id,
                           organizations,
                           contributors,
                           sum(committee_total) AS total,
                           cmte_id,
                           pacshort AS committee
                    FROM
                      (SELECT match_id,
                              organizations,
                              contributors,
                              sum(amount) AS committee_total,
                              cmteid AS cmte_id
                       FROM
                         (SELECT CASE
                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                     ELSE orgname
                                 END AS match_id,
                                 array_agg(DISTINCT orgname) AS organizations,
                                 array_agg(DISTINCT contrib) AS contributors
                          FROM crp_contributions
                          LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
                          AND crp_committees.cycle = '""" + reference_cycle + """'
                          WHERE primcode IN ('J1100',
                                             'J2200',
                                             'J2400',
                                             'Z1100',
                                             'Z4100',
                                             'Z4500',
                                             'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
                            AND TYPE NOT IN ('11J',
                                             '15C',
                                             '15E',
                                             '15I',
                                             '15J',
                                             '15T',
                                             '18J',
                                             '19J',
                                             '30J',
                                             '30F',
                                             '31J',
                                             '31F',
                                             '32J',
                                             '32F')
                          GROUP BY match_id
                          HAVING sum(amount) >= 50000) AS gop_donors
                       JOIN crp_contributions ON CASE
                                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                                     ELSE crp_contributions.orgname
                                                 END = gop_donors.match_id
                       WHERE date >= '""" + start_one + """'
                         AND date <= '""" + end_one + """'
                         or (date >= '""" + start_two + """'
                         and date <= '""" + end_two + """')
                         AND CYCLE = '""" + cycle + """'
                         AND TYPE NOT IN ('11J',
                                          '15C',
                                          '15E',
                                          '15I',
                                          '15J',
                                          '15T',
                                          '18J',
                                          '19J',
                                          '30J',
                                          '30F',
                                          '31J',
                                          '31F',
                                          '32J',
                                          '32F')
                       GROUP BY match_id,
                                organizations,
                                contributors,
                                cmte_id) AS donors_committees
                    JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
                    WHERE primcode IN ('J1100',
                                       'J2200',
                                       'J2400',
                                       'Z1100',
                                       'Z4100',
                                       'Z4500',
                                       'Z5100')
                    GROUP BY match_id,
                             organizations,
                             contributors,
                             cmte_id,
                             committee;

                    SELECT *
                    FROM """+prefix+"""_donors_committees_"""+suffix+""";
                    """
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [61]:
inside_bill_donors_committees_17_1st_qtr = run_donors_query_1st_qtr("inside", "17_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2018", "2017-02-02", "2017-03-31", "2017-02-02", "2017-03-31")
outside_bill_donors_committees_17_1st_qtr = run_donors_query_1st_qtr("outside", "17_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2018", "2017-01-01", "2017-02-01", "2017-04-01", "2017-12-31")

In [62]:
inside_bill_donors_committees_15_1st_qtr = run_donors_query_1st_qtr("inside", "15_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2016", "2015-02-02", "2015-03-31", "2015-02-02", "2015-03-31")
outside_bill_donors_committees_15_1st_qtr = run_donors_query_1st_qtr("outside", "15_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2016", "2015-01-01", "2015-02-01", "2015-04-01", "2015-12-31")

In [63]:
inside_bill_donors_committees_13_1st_qtr = run_donors_query_1st_qtr("inside", "13_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2014", "2013-02-02", "2013-03-31", "2013-02-02", "2013-03-31")
outside_bill_donors_committees_13_1st_qtr = run_donors_query_1st_qtr("outside", "13_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2014", "2013-01-01", "2013-02-01", "2013-04-01", "2013-12-31")

In [64]:
inside_bill_donors_committees_11_1st_qtr = run_donors_query_1st_qtr("inside", "11_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2012", "2011-02-02", "2011-03-31", "2011-02-02", "2011-03-31")
outside_bill_donors_committees_11_1st_qtr = run_donors_query_1st_qtr("outside", "11_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2012", "2011-01-01", "2011-02-01", "2011-04-01", "2011-12-31")

In [65]:
inside_bill_donors_committees_09_1st_qtr = run_donors_query_1st_qtr("inside", "09_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2010", "2009-02-02", "2009-03-31", "2009-02-02", "2009-03-31")
outside_bill_donors_committees_09_1st_qtr = run_donors_query_1st_qtr("outside", "09_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2010", "2009-01-01", "2009-02-01", "2009-04-01", "2009-12-31")

## How much did each contributor give in each period for each cycle?

### 2017

In [66]:
inside_bill_donors_17_1st_qtr = inside_bill_donors_committees_17_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_17_1st_qtr = outside_bill_donors_committees_17_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [67]:
donors_17_1st_qtr = inside_bill_donors_17_1st_qtr.merge(outside_bill_donors_17_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_17_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_17_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_17_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 5 columns):
match_id              269 non-null object
organizations         269 non-null object
contributors          269 non-null object
total_inside_bill     269 non-null float64
total_outside_bill    269 non-null float64
dtypes: float64(2), object(3)
memory usage: 12.6+ KB


In [68]:
donors_17_1st_qtr["pct_inside_bill"] = donors_17_1st_qtr["total_inside_bill"] / (donors_17_1st_qtr["total_outside_bill"] + donors_17_1st_qtr["total_inside_bill"])
donors_17_1st_qtr["pct_outside_bill"] = donors_17_1st_qtr["total_outside_bill"] / (donors_17_1st_qtr["total_outside_bill"] + donors_17_1st_qtr["total_inside_bill"])
donors_17_1st_qtr["change"] = donors_17_1st_qtr["total_inside_bill"] - donors_17_1st_qtr["total_outside_bill"]
donors_17_1st_qtr["pct_change"] = (donors_17_1st_qtr["total_inside_bill"] - donors_17_1st_qtr["total_outside_bill"]) / donors_17_1st_qtr["total_outside_bill"].abs()
donors_17_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
2,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],"3,592,631.00","9,297,414.00",0.28,0.72,"-5,704,783.00",-0.61
69,U0000003690,[Uline Inc],"[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH A, UIH...","2,076,600.00","14,552,500.00",0.12,0.88,"-12,475,900.00",-0.86
1,Ai Altep Holdings,[Ai Altep Holdings],[AI ALTEP HOLDINGS INC],"1,000,000.00",0.00,1.00,0.00,"1,000,000.00",inf
38,U0000003265,"[Dimensional Fund Advisors, Show Me Institute]","[SINQUEFIELD, JEANNE C MRS, SINQUEFIELD, REX A]","968,600.00","99,800.00",0.91,0.09,"868,800.00",8.71
4,Chevron Corp,[Chevron Corp],"[CHEVRON, CHEVRON POLICY GOVERNMENT & PUBLIC A...","950,000.00","750,000.00",0.56,0.44,"200,000.00",0.27


#### How much did they give?

In [69]:
print("2017 inside-bill 1st quarter total: $" + str(donors_17_1st_qtr["total_inside_bill"].sum()))
print("2017 outside-bill 1st quarter total: $" + str(donors_17_1st_qtr["total_outside_bill"].sum()))

2017 inside-bill 1st quarter total: $52662234.0
2017 outside-bill 1st quarter total: $68899644.0


### 2015

In [70]:
inside_bill_donors_15_1st_qtr = inside_bill_donors_committees_15_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_15_1st_qtr = outside_bill_donors_committees_15_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [71]:
donors_15_1st_qtr = inside_bill_donors_15_1st_qtr.merge(outside_bill_donors_15_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_15_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_15_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_15_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_15_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245 entries, 0 to 244
Data columns (total 5 columns):
match_id              245 non-null object
organizations         192 non-null object
contributors          192 non-null object
total_inside_bill     192 non-null float64
total_outside_bill    245 non-null float64
dtypes: float64(2), object(3)
memory usage: 11.5+ KB


In [72]:
donors_15_1st_qtr["pct_inside_bill"] = donors_15_1st_qtr["total_inside_bill"] / (donors_15_1st_qtr["total_outside_bill"] + donors_15_1st_qtr["total_inside_bill"])
donors_15_1st_qtr["pct_outside_bill"] = donors_15_1st_qtr["total_outside_bill"] / (donors_15_1st_qtr["total_outside_bill"] + donors_15_1st_qtr["total_inside_bill"])
donors_15_1st_qtr["change"] = donors_15_1st_qtr["total_inside_bill"] - donors_15_1st_qtr["total_outside_bill"]
donors_15_1st_qtr["pct_change"] = (donors_15_1st_qtr["total_inside_bill"] - donors_15_1st_qtr["total_outside_bill"]) / donors_15_1st_qtr["total_outside_bill"].abs()
donors_15_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
17,U0000002708,[Mountaire Corp],"[CAMERON, RONALD, CAMERON, RONALD M MR]","3,012,700.00","191,500.00",0.94,0.06,"2,821,200.00",14.73
68,U0000004014,"[MBF Healthcare Partners, Mindful Kids Miami]","[FERNANDEZ, CONSTANCE, FERNANDEZ, CONSTANCE M,...","1,075,520.00","2,402,100.00",0.31,0.69,"-1,326,580.00",-0.55
53,U0000003658,"[Franklin Resources, Psychiatrist]","[JOHNSON, ANN L, JOHNSON, CHARLES, JOHNSON, CH...","1,029,600.00","663,900.00",0.61,0.39,"365,700.00",0.55
2,NextEra Energy,[NextEra Energy],[NEXTERA ENERGY],"1,025,000.00","250,000.00",0.80,0.20,"775,000.00",3.10
55,U0000003682,[Renaissance Technologies],"[MERCER, DIANA L MRS, MERCER, DIANA L MS, MERC...","754,600.00","14,323,000.00",0.05,0.95,"-13,568,400.00",-0.95


#### How much did they give?

In [73]:
print("2015 inside-bill 1st quarter total: $" + str(donors_15_1st_qtr["total_inside_bill"].sum()))
print("2015 outside-bill 1st quarter total: $" + str(donors_15_1st_qtr["total_outside_bill"].sum()))

2015 inside-bill 1st quarter total: $20592227.0
2015 outside-bill 1st quarter total: $103676230.0


### 2013

In [74]:
inside_bill_donors_13_1st_qtr = inside_bill_donors_committees_13_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_13_1st_qtr = outside_bill_donors_committees_13_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [75]:
donors_13_1st_qtr = inside_bill_donors_13_1st_qtr.merge(outside_bill_donors_13_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_13_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_13_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_13_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_13_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 0 to 213
Data columns (total 5 columns):
match_id              214 non-null object
organizations         131 non-null object
contributors          131 non-null object
total_inside_bill     131 non-null float64
total_outside_bill    214 non-null float64
dtypes: float64(2), object(3)
memory usage: 10.0+ KB


In [76]:
donors_13_1st_qtr["pct_inside_bill"] = donors_13_1st_qtr["total_inside_bill"] / (donors_13_1st_qtr["total_outside_bill"] + donors_13_1st_qtr["total_inside_bill"])
donors_13_1st_qtr["pct_outside_bill"] = donors_13_1st_qtr["total_outside_bill"] / (donors_13_1st_qtr["total_outside_bill"] + donors_13_1st_qtr["total_inside_bill"])
donors_13_1st_qtr["change"] = donors_13_1st_qtr["total_inside_bill"] - donors_13_1st_qtr["total_outside_bill"]
donors_13_1st_qtr["pct_change"] = (donors_13_1st_qtr["total_inside_bill"] - donors_13_1st_qtr["total_outside_bill"]) / donors_13_1st_qtr["total_outside_bill"].abs()
donors_13_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
60,U0000004145,[Tommar LLC],"[LARKIN JR, THOMAS E, LARKIN, MARGARET, LARKIN...","210,400.00","40,600.00",0.84,0.16,"169,800.00",4.18
45,U0000003690,[Uline Inc],"[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH A, UIH...","203,700.00","590,850.00",0.26,0.74,"-387,150.00",-0.66
32,U0000003558,[Satter Investment Management],"[SATTER, MUNEER A]","175,200.00","187,800.00",0.48,0.52,"-12,600.00",-0.07
63,U0000004227,"[Kinder Foundation, Kinder Morgan Inc]","[KINDER, NANCY G, KINDER, NANCY G MR, KINDER, ...","165,900.00","10,000.00",0.94,0.06,"155,900.00",15.59
43,U0000003658,"[Franklin Resources, Psychiatrist]","[JOHNSON, ANN L, JOHNSON, CHARLES, JOHNSON, CH...","155,200.00","11,066.00",0.93,0.07,"144,134.00",13.02


#### How much did they give?

In [77]:
print("2013 inside-bill 1st quarter total: $" + str(donors_13_1st_qtr["total_inside_bill"].sum()))
print("2013 outside-bill 1st quarter total: $" + str(donors_13_1st_qtr["total_outside_bill"].sum()))

2013 inside-bill 1st quarter total: $4389003.0
2013 outside-bill 1st quarter total: $16110304.0


### 2011

In [78]:
inside_bill_donors_11_1st_qtr = inside_bill_donors_committees_11_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_11_1st_qtr = outside_bill_donors_committees_11_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [79]:
donors_11_1st_qtr = inside_bill_donors_11_1st_qtr.merge(outside_bill_donors_11_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_11_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_11_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_11_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_11_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 5 columns):
match_id              217 non-null object
organizations         130 non-null object
contributors          130 non-null object
total_inside_bill     130 non-null float64
total_outside_bill    217 non-null float64
dtypes: float64(2), object(3)
memory usage: 10.2+ KB


In [80]:
donors_11_1st_qtr["pct_inside_bill"] = donors_11_1st_qtr["total_inside_bill"] / (donors_11_1st_qtr["total_outside_bill"] + donors_11_1st_qtr["total_inside_bill"])
donors_11_1st_qtr["pct_outside_bill"] = donors_11_1st_qtr["total_outside_bill"] / (donors_11_1st_qtr["total_outside_bill"] + donors_11_1st_qtr["total_inside_bill"])
donors_11_1st_qtr["change"] = donors_11_1st_qtr["total_inside_bill"] - donors_11_1st_qtr["total_outside_bill"]
donors_11_1st_qtr["pct_change"] = (donors_11_1st_qtr["total_inside_bill"] - donors_11_1st_qtr["total_outside_bill"]) / donors_11_1st_qtr["total_outside_bill"].abs()
donors_11_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
54,U0000003932,"[[24I Contribution], Research Affiliates LLC]","[ARNOTT, ROBERT D, ARNOTT, ROBERT D MR]","267,600.00","312,500.00",0.46,0.54,"-44,900.00",-0.14
34,U0000003558,[Satter Investment Management],"[SATTER, MUNEER A]","150,000.00","289,000.00",0.34,0.66,"-139,000.00",-0.48
38,U0000003614,[PVS Chemicals],"[NICHOLSON, JAMES B]","117,000.00","69,000.00",0.63,0.37,"48,000.00",0.70
39,U0000003616,[Centra Inc],"[MOROUN, MATTHEW, MOROUN, MATTHEW T MR]","92,600.00","71,300.00",0.56,0.44,"21,300.00",0.30
17,U0000003246,[Jim Click Automotive],"[CLICK, JIM, CLICK, JIM JR, CLICK, VICKI, CLIC...","74,100.00","93,300.00",0.44,0.56,"-19,200.00",-0.21


#### How much did they give?

In [81]:
print("2011 inside-bill 1st quarter total: $" + str(donors_11_1st_qtr["total_inside_bill"].sum()))
print("2011 outside-bill 1st quarter total: $" + str(donors_11_1st_qtr["total_outside_bill"].sum()))

2011 inside-bill 1st quarter total: $3032290.0
2011 outside-bill 1st quarter total: $18837562.0


### 2009

In [82]:
inside_bill_donors_09_1st_qtr = inside_bill_donors_committees_09_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_09_1st_qtr = outside_bill_donors_committees_09_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [83]:
donors_09_1st_qtr = inside_bill_donors_09_1st_qtr.merge(outside_bill_donors_09_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_09_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_09_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_09_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_09_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 0 to 180
Data columns (total 5 columns):
match_id              181 non-null object
organizations         107 non-null object
contributors          107 non-null object
total_inside_bill     107 non-null float64
total_outside_bill    181 non-null float64
dtypes: float64(2), object(3)
memory usage: 8.5+ KB


In [84]:
donors_09_1st_qtr["pct_inside_bill"] = donors_09_1st_qtr["total_inside_bill"] / (donors_09_1st_qtr["total_outside_bill"] + donors_09_1st_qtr["total_inside_bill"])
donors_09_1st_qtr["pct_outside_bill"] = donors_09_1st_qtr["total_outside_bill"] / (donors_09_1st_qtr["total_outside_bill"] + donors_09_1st_qtr["total_inside_bill"])
donors_09_1st_qtr["change"] = donors_09_1st_qtr["total_inside_bill"] - donors_09_1st_qtr["total_outside_bill"]
donors_09_1st_qtr["pct_change"] = (donors_09_1st_qtr["total_inside_bill"] - donors_09_1st_qtr["total_outside_bill"]) / donors_09_1st_qtr["total_outside_bill"].abs()
donors_09_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
22,U0000003295,"[[24T Contribution], Richard & Barbara Gaby Fo...","[GABY, BARBARA, GABY, RICHARD, GABY, RICHARD D...","90,000.00","62,800.00",0.59,0.41,"27,200.00",0.43
7,U0000000280,[Crow Holdings],"[CROW, HARLAN, CROW, HARLAN MR, CROW, HARLAN R...","80,600.00","176,800.00",0.31,0.69,"-96,200.00",-0.54
16,U0000003246,[Jim Click Automotive],"[CLICK, JIM, CLICK, JIM JR, CLICK, VICKI, CLIC...","65,800.00","19,400.00",0.77,0.23,"46,400.00",2.39
20,U0000003292,[Cintas Corp],"[FARMER, JOYCE E, FARMER, RICHARD T, FARMER, R T]","62,300.00","88,000.00",0.41,0.59,"-25,700.00",-0.29
21,U0000003293,[Sable Minerals],"[FLORES, CHERIE H, FLORES, CHERIE H MRS, FLORE...","60,800.00","18,700.00",0.76,0.24,"42,100.00",2.25


In [85]:
print("2009 inside-bill 1st quarter total: $" + str(donors_09_1st_qtr["total_inside_bill"].sum()))
print("2009 outside-bill 1st quarter total: $" + str(donors_09_1st_qtr["total_outside_bill"].sum()))

2009 inside-bill 1st quarter total: $1403605.0
2009 outside-bill 1st quarter total: $5133871.0


## How did each contributor's giving pattern change in each cycle compared with 2017?

### 2015

In [86]:
donors_15_17_1st_qtr = donors_17_1st_qtr.merge(donors_15_1st_qtr, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17_1st_qtr.drop(["change_17", "pct_change_17", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17_1st_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_15_17_1st_qtr = donors_15_17_1st_qtr[["match_id", "contributors", "organizations", "total_outside_bill_15", "total_inside_bill_15",
                 "pct_outside_bill_15", "pct_inside_bill_15", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_15_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 11 columns):
match_id                 269 non-null object
contributors             269 non-null object
organizations            269 non-null object
total_outside_bill_15    245 non-null float64
total_inside_bill_15     192 non-null float64
pct_outside_bill_15      192 non-null float64
pct_inside_bill_15       192 non-null float64
total_outside_bill_17    269 non-null float64
total_inside_bill_17     269 non-null float64
pct_outside_bill_17      269 non-null float64
pct_inside_bill_17       269 non-null float64
dtypes: float64(8), object(3)
memory usage: 25.2+ KB


In [87]:
donors_15_17_1st_qtr["giving_change"] = np.where((donors_15_17_1st_qtr["pct_inside_bill_17"] > donors_15_17_1st_qtr["pct_inside_bill_15"]) | (donors_15_17_1st_qtr["pct_outside_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17_1st_qtr["pct_inside_bill_17"] < donors_15_17_1st_qtr["pct_inside_bill_15"], "Decreased",
                                           np.where(donors_15_17_1st_qtr["pct_inside_bill_17"] == donors_15_17_1st_qtr["pct_inside_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17_1st_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_15,total_inside_bill_15,pct_outside_bill_15,pct_inside_bill_15,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Agua Caliente Band of Cahuilla Indians,"[AGUA CALIENTE BAND OF CAHUILLA, AGUA CALIENTE...",[Agua Caliente Band of Cahuilla Indians],"83,500.00","50,400.00",0.62,0.38,"198,300.00","57,900.00",0.77,0.23,Decreased


### 2013

In [88]:
donors_13_17_1st_qtr = donors_17_1st_qtr.merge(donors_13_1st_qtr, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17_1st_qtr.drop(["change_17", "pct_change_17", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17_1st_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_13_17_1st_qtr = donors_13_17_1st_qtr[["match_id", "contributors", "organizations", "total_outside_bill_13", "total_inside_bill_13",
                 "pct_outside_bill_13", "pct_inside_bill_13", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_13_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 11 columns):
match_id                 269 non-null object
contributors             269 non-null object
organizations            269 non-null object
total_outside_bill_13    214 non-null float64
total_inside_bill_13     131 non-null float64
pct_outside_bill_13      131 non-null float64
pct_inside_bill_13       131 non-null float64
total_outside_bill_17    269 non-null float64
total_inside_bill_17     269 non-null float64
pct_outside_bill_17      269 non-null float64
pct_inside_bill_17       269 non-null float64
dtypes: float64(8), object(3)
memory usage: 25.2+ KB


In [89]:
donors_13_17_1st_qtr["giving_change"] = np.where((donors_13_17_1st_qtr["pct_inside_bill_17"] > donors_13_17_1st_qtr["pct_inside_bill_13"]) | (donors_13_17_1st_qtr["pct_outside_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17_1st_qtr["pct_inside_bill_17"] < donors_13_17_1st_qtr["pct_inside_bill_13"], "Decreased",
                                           np.where(donors_13_17_1st_qtr["pct_inside_bill_17"] == donors_13_17_1st_qtr["pct_inside_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17_1st_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_13,total_inside_bill_13,pct_outside_bill_13,pct_inside_bill_13,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Agua Caliente Band of Cahuilla Indians,"[AGUA CALIENTE BAND OF CAHUILLA, AGUA CALIENTE...",[Agua Caliente Band of Cahuilla Indians],"29,450.00","77,600.00",0.28,0.72,"198,300.00","57,900.00",0.77,0.23,Decreased


### 2011

In [90]:
donors_11_17_1st_qtr = donors_17_1st_qtr.merge(donors_11_1st_qtr, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17_1st_qtr.drop(["change_17", "pct_change_17", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17_1st_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_11_17_1st_qtr = donors_11_17_1st_qtr[["match_id", "contributors", "organizations", "total_outside_bill_11", "total_inside_bill_11",
                 "pct_outside_bill_11", "pct_inside_bill_11", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_11_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 11 columns):
match_id                 269 non-null object
contributors             269 non-null object
organizations            269 non-null object
total_outside_bill_11    217 non-null float64
total_inside_bill_11     130 non-null float64
pct_outside_bill_11      130 non-null float64
pct_inside_bill_11       130 non-null float64
total_outside_bill_17    269 non-null float64
total_inside_bill_17     269 non-null float64
pct_outside_bill_17      269 non-null float64
pct_inside_bill_17       269 non-null float64
dtypes: float64(8), object(3)
memory usage: 25.2+ KB


In [91]:
donors_11_17_1st_qtr["giving_change"] = np.where((donors_11_17_1st_qtr["pct_inside_bill_17"] > donors_11_17_1st_qtr["pct_inside_bill_11"]) | (donors_11_17_1st_qtr["pct_outside_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17_1st_qtr["pct_inside_bill_17"] < donors_11_17_1st_qtr["pct_inside_bill_11"], "Decreased",
                                           np.where(donors_11_17_1st_qtr["pct_inside_bill_17"] == donors_11_17_1st_qtr["pct_inside_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17_1st_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_11,total_inside_bill_11,pct_outside_bill_11,pct_inside_bill_11,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Agua Caliente Band of Cahuilla Indians,"[AGUA CALIENTE BAND OF CAHUILLA, AGUA CALIENTE...",[Agua Caliente Band of Cahuilla Indians],"53,800.00","2,000.00",0.96,0.04,"198,300.00","57,900.00",0.77,0.23,Increased


### 2009

In [92]:
donors_09_17_1st_qtr = donors_17_1st_qtr.merge(donors_09_1st_qtr, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17_1st_qtr.drop(["change_17", "pct_change_17", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17_1st_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_09_17_1st_qtr = donors_09_17_1st_qtr[["match_id", "contributors", "organizations", "total_outside_bill_09", "total_inside_bill_09",
                 "pct_outside_bill_09", "pct_inside_bill_09", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_09_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 11 columns):
match_id                 269 non-null object
contributors             269 non-null object
organizations            269 non-null object
total_outside_bill_09    181 non-null float64
total_inside_bill_09     107 non-null float64
pct_outside_bill_09      107 non-null float64
pct_inside_bill_09       107 non-null float64
total_outside_bill_17    269 non-null float64
total_inside_bill_17     269 non-null float64
pct_outside_bill_17      269 non-null float64
pct_inside_bill_17       269 non-null float64
dtypes: float64(8), object(3)
memory usage: 25.2+ KB


In [93]:
donors_09_17_1st_qtr["giving_change"] = np.where((donors_09_17_1st_qtr["pct_inside_bill_17"] > donors_09_17_1st_qtr["pct_inside_bill_09"]) | (donors_09_17_1st_qtr["pct_outside_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17_1st_qtr["pct_inside_bill_17"] < donors_09_17_1st_qtr["pct_inside_bill_09"], "Decreased",
                                           np.where(donors_09_17_1st_qtr["pct_inside_bill_17"] == donors_09_17_1st_qtr["pct_inside_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17_1st_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_09,total_inside_bill_09,pct_outside_bill_09,pct_inside_bill_09,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Agua Caliente Band of Cahuilla Indians,"[AGUA CALIENTE BAND OF CAHUILLA, AGUA CALIENTE...",[Agua Caliente Band of Cahuilla Indians],"6,000.00","30,400.00",0.16,0.84,"198,300.00","57,900.00",0.77,0.23,Decreased


## What proportion of donors increased the share of their annual giving that fell within the two-month period in 2017 as compared with the same period in each of the previous years?

In [94]:
print(str(donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_15_17_1st_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2015 and 2017.")
print(str(donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_13_17_1st_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2013 and 2017.")
print(str(donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_11_17_1st_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2011 and 2017.")
print(str(donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_09_17_1st_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2009 and 2017.")

87.36059479553904% of donors increased the share of their annual giving that fell within the two-month period between 2015 and 2017.
88.10408921933085% of donors increased the share of their annual giving that fell within the two-month period between 2013 and 2017.
94.42379182156134% of donors increased the share of their annual giving that fell within the two-month period between 2011 and 2017.
89.59107806691449% of donors increased the share of their annual giving that fell within the two-month period between 2009 and 2017.


## And by how much did these individuals' giving increase?

In [95]:
print("These donors' giving within the two-month period increased by $" + str(donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_15"].sum()) + ", from $" + str(donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_15"].sum()) + " to $" + str(donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2015 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_13"].sum()) + ", from $" + str(donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_13"].sum()) + " to $" + str(donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2013 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_11"].sum()) + ", from $" + str(donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_11"].sum()) + " to $" + str(donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2011 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_09"].sum()) + ", from $" + str(donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_09"].sum()) + " to $" + str(donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2009 and 2017.")

These donors' giving within the two-month period increased by $36607174.0, from $11774527.0 to $48381701.0 between 2015 and 2017.
These donors' giving within the two-month period increased by $43518300.0, from $2505309.0 to $46023609.0 between 2013 and 2017.
These donors' giving within the two-month period increased by $49030814.0, from $2286890.0 to $51317704.0 between 2011 and 2017.
These donors' giving within the two-month period increased by $46863046.0, from $744955.0 to $47608001.0 between 2009 and 2017.


## How many donors in each year gave all of their money in the two-month period of each year?

In [96]:
print("In 2017, " + str(donors_17_1st_qtr[donors_17_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2015, " + str(donors_15_1st_qtr[donors_15_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2013, " + str(donors_13_1st_qtr[donors_13_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2011, " + str(donors_11_1st_qtr[donors_11_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2009, " + str(donors_09_1st_qtr[donors_09_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")

In 2017, 43 donors gave all their money in the two-month period.
In 2015, 6 donors gave all their money in the two-month period.
In 2013, 5 donors gave all their money in the two-month period.
In 2011, 1 donors gave all their money in the two-month period.
In 2009, 8 donors gave all their money in the two-month period.


# What does this giving look like on a contribution-by-contribution level?

In [97]:
def run_contributions_query_1st_qtr(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start_one, end_one, start_two, end_two):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_contributions_""" + suffix + """ AS
    SELECT fectransid,
    match_id,
    organization,
    contributor, date, sum(SUM),
    cmte_id,
    pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
                       WHERE date >= '""" + start_one + """'
                         AND date <= '""" + end_one + """'
                         or (date >= '""" + start_two + """'
                         and date <= '""" + end_two + """')
                         AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;
                            SELECT *
                            FROM """ + prefix + """_contributions_"""+ suffix +""";"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [98]:
inside_bill_contributions_17_1st_qtr = run_contributions_query_1st_qtr("inside", "17_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2018", "2017-02-02", "2017-03-31", "2017-02-02", "2017-03-31")
outside_bill_contributions_17_1st_qtr = run_contributions_query_1st_qtr("outside", "17_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2018", "2017-01-01", "2017-02-01", "2017-04-01", "2017-12-31")

In [99]:
inside_bill_contributions_15_1st_qtr = run_contributions_query_1st_qtr("inside", "15_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2016", "2015-02-02", "2015-03-31", "2015-02-02", "2015-03-31")
outside_bill_contributions_15_1st_qtr = run_contributions_query_1st_qtr("outside", "15_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2016", "2015-01-01", "2015-02-01", "2015-04-01", "2015-12-31")

In [100]:
inside_bill_contributions_13_1st_qtr = run_contributions_query_1st_qtr("inside", "13_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2014", "2013-02-02", "2013-03-31", "2013-02-02", "2013-03-31")
outside_bill_contributions_13_1st_qtr = run_contributions_query_1st_qtr("outside", "13_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2014", "2013-01-01", "2013-02-01", "2013-04-01", "2013-12-31")

In [101]:
inside_bill_contributions_11_1st_qtr = run_contributions_query_1st_qtr("inside", "11_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2012", "2011-02-02", "2011-03-31", "2011-02-02", "2011-03-31")
outside_bill_contributions_11_1st_qtr = run_contributions_query_1st_qtr("outside", "11_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2012", "2011-01-01", "2011-02-01", "2011-04-01", "2011-12-31")

In [102]:
inside_bill_contributions_09_1st_qtr = run_contributions_query_1st_qtr("inside", "09_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2010", "2009-02-02", "2009-03-31", "2009-02-02", "2009-03-31")
outside_bill_contributions_09_1st_qtr = run_contributions_query_1st_qtr("outside", "09_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2010", "2009-01-01", "2009-02-01", "2009-04-01", "2009-12-31")

## Return the donors who gave at least $50,000 to conservative and GOP-aligned groups between May 2 and June 30, 2017, split by their giving inside and outside of that period. Do this for every off-year going back to the 2010 cycle.

In [103]:
def run_donors_query_2nd_qtr(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start_one, end_one, start_two, end_two):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """+prefix+"""_donors_committees_"""+suffix+""" AS
                    SELECT match_id,
                           organizations,
                           contributors,
                           sum(committee_total) AS total,
                           cmte_id,
                           pacshort AS committee
                    FROM
                      (SELECT match_id,
                              organizations,
                              contributors,
                              sum(amount) AS committee_total,
                              cmteid AS cmte_id
                       FROM
                         (SELECT CASE
                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                     ELSE orgname
                                 END AS match_id,
                                 array_agg(DISTINCT orgname) AS organizations,
                                 array_agg(DISTINCT contrib) AS contributors
                          FROM crp_contributions
                          LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
                          AND crp_committees.cycle = '""" + reference_cycle + """'
                          WHERE primcode IN ('J1100',
                                             'J2200',
                                             'J2400',
                                             'Z1100',
                                             'Z4100',
                                             'Z4500',
                                             'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
                            AND TYPE NOT IN ('11J',
                                             '15C',
                                             '15E',
                                             '15I',
                                             '15J',
                                             '15T',
                                             '18J',
                                             '19J',
                                             '30J',
                                             '30F',
                                             '31J',
                                             '31F',
                                             '32J',
                                             '32F')
                          GROUP BY match_id
                          HAVING sum(amount) >= 50000) AS gop_donors
                       JOIN crp_contributions ON CASE
                                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                                     ELSE crp_contributions.orgname
                                                 END = gop_donors.match_id
                       WHERE date >= '""" + start_one + """'
                         AND date <= '""" + end_one + """'
                         or (date >= '""" + start_two + """'
                         and date <= '""" + end_two + """')
                         AND CYCLE = '""" + cycle + """'
                         AND TYPE NOT IN ('11J',
                                          '15C',
                                          '15E',
                                          '15I',
                                          '15J',
                                          '15T',
                                          '18J',
                                          '19J',
                                          '30J',
                                          '30F',
                                          '31J',
                                          '31F',
                                          '32J',
                                          '32F')
                       GROUP BY match_id,
                                organizations,
                                contributors,
                                cmte_id) AS donors_committees
                    JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
                    WHERE primcode IN ('J1100',
                                       'J2200',
                                       'J2400',
                                       'Z1100',
                                       'Z4100',
                                       'Z4500',
                                       'Z5100')
                    GROUP BY match_id,
                             organizations,
                             contributors,
                             cmte_id,
                             committee;

                    SELECT *
                    FROM """+prefix+"""_donors_committees_"""+suffix+""";
                    """
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [104]:
inside_bill_donors_committees_17_2nd_qtr = run_donors_query_2nd_qtr("inside", "17_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2018", "2017-05-02", "2017-06-30", "2017-05-02", "2017-06-30")
outside_bill_donors_committees_17_2nd_qtr = run_donors_query_2nd_qtr("outside", "17_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2018", "2017-01-01", "2017-05-01", "2017-07-01", "2017-12-31")

In [105]:
inside_bill_donors_committees_15_2nd_qtr = run_donors_query_2nd_qtr("inside", "15_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2016", "2015-05-02", "2015-06-30", "2015-05-02", "2015-06-30")
outside_bill_donors_committees_15_2nd_qtr = run_donors_query_2nd_qtr("outside", "15_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2016", "2015-01-01", "2015-05-01", "2015-07-01", "2015-12-31")

In [106]:
inside_bill_donors_committees_13_2nd_qtr = run_donors_query_2nd_qtr("inside", "13_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2014", "2013-05-02", "2013-06-30", "2013-05-02", "2013-06-30")
outside_bill_donors_committees_13_2nd_qtr = run_donors_query_2nd_qtr("outside", "13_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2014", "2013-01-01", "2013-05-01", "2013-07-01", "2013-12-31")

In [107]:
inside_bill_donors_committees_11_2nd_qtr = run_donors_query_2nd_qtr("inside", "11_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2012", "2011-05-02", "2011-06-30", "2011-05-02", "2011-06-30")
outside_bill_donors_committees_11_2nd_qtr = run_donors_query_2nd_qtr("outside", "11_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2012", "2011-01-01", "2011-05-01", "2011-07-01", "2011-12-31")

In [108]:
inside_bill_donors_committees_09_2nd_qtr = run_donors_query_2nd_qtr("inside", "09_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2010", "2009-05-02", "2009-06-30", "2009-05-02", "2009-06-30")
outside_bill_donors_committees_09_2nd_qtr = run_donors_query_2nd_qtr("outside", "09_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2010", "2009-01-01", "2009-05-01", "2009-07-01", "2009-12-31")

## How much did each contributor give in each period for each cycle?

### 2017

In [109]:
inside_bill_donors_17_2nd_qtr = inside_bill_donors_committees_17_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_17_2nd_qtr = outside_bill_donors_committees_17_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [110]:
donors_17_2nd_qtr = inside_bill_donors_17_2nd_qtr.merge(outside_bill_donors_17_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_17_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_17_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_17_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219 entries, 0 to 218
Data columns (total 5 columns):
match_id              219 non-null object
organizations         219 non-null object
contributors          219 non-null object
total_inside_bill     219 non-null float64
total_outside_bill    219 non-null float64
dtypes: float64(2), object(3)
memory usage: 10.3+ KB


In [111]:
donors_17_2nd_qtr["pct_inside_bill"] = donors_17_2nd_qtr["total_inside_bill"] / (donors_17_2nd_qtr["total_outside_bill"] + donors_17_2nd_qtr["total_inside_bill"])
donors_17_2nd_qtr["pct_outside_bill"] = donors_17_2nd_qtr["total_outside_bill"] / (donors_17_2nd_qtr["total_outside_bill"] + donors_17_2nd_qtr["total_inside_bill"])
donors_17_2nd_qtr["change"] = donors_17_2nd_qtr["total_inside_bill"] - donors_17_2nd_qtr["total_outside_bill"]
donors_17_2nd_qtr["pct_change"] = (donors_17_2nd_qtr["total_inside_bill"] - donors_17_2nd_qtr["total_outside_bill"]) / donors_17_2nd_qtr["total_outside_bill"].abs()
donors_17_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
54,U0000003690,"[[24T Contribution], Uline Inc, None]","[UIHLEIN, RICHARD, UIHLEIN, RICHARD E, UIHLEIN...","3,263,500.00","13,454,300.00",0.20,0.80,"-10,190,800.00",-0.76
16,U0000000073,[Western Refining],"[FOSTER, PAUL L]","1,495,400.00","308,400.00",0.83,0.17,"1,187,000.00",3.85
118,d0000350730,[Aileron],"[MATHILE, CLAYTON]","1,000,000.00",0.00,1.00,0.00,"1,000,000.00",inf
53,U0000003682,[Renaissance Technologies],"[MERCER, DIANA L MS, MERCER, ROBERT, MERCER, R...","868,000.00","3,297,100.00",0.21,0.79,"-2,429,100.00",-0.74
26,U0000002706,"[[24T Contribution], Ariel Corp]","[BUCHWALD-WRIGHT, KAREN MS, WRIGHT, KAREN, WRI...","730,216.00","738,600.00",0.50,0.50,"-8,384.00",-0.01


#### How much did they give?

In [112]:
print("2017 inside-bill 2nd quarter total: $" + str(donors_17_2nd_qtr["total_inside_bill"].sum()))
print("2017 outside-bill 2nd quarter total: $" + str(donors_17_2nd_qtr["total_outside_bill"].sum()))

2017 inside-bill 2nd quarter total: $43698413.0
2017 outside-bill 2nd quarter total: $70552000.0


### 2015

In [113]:
inside_bill_donors_15_2nd_qtr = inside_bill_donors_committees_15_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_15_2nd_qtr = outside_bill_donors_committees_15_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [114]:
donors_15_2nd_qtr = inside_bill_donors_15_2nd_qtr.merge(outside_bill_donors_15_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_15_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_15_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_15_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_15_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186 entries, 0 to 185
Data columns (total 5 columns):
match_id              186 non-null object
organizations         149 non-null object
contributors          149 non-null object
total_inside_bill     149 non-null float64
total_outside_bill    186 non-null float64
dtypes: float64(2), object(3)
memory usage: 8.7+ KB


In [115]:
donors_15_2nd_qtr["pct_inside_bill"] = donors_15_2nd_qtr["total_inside_bill"] / (donors_15_2nd_qtr["total_outside_bill"] + donors_15_2nd_qtr["total_inside_bill"])
donors_15_2nd_qtr["pct_outside_bill"] = donors_15_2nd_qtr["total_outside_bill"] / (donors_15_2nd_qtr["total_outside_bill"] + donors_15_2nd_qtr["total_inside_bill"])
donors_15_2nd_qtr["change"] = donors_15_2nd_qtr["total_inside_bill"] - donors_15_2nd_qtr["total_outside_bill"]
donors_15_2nd_qtr["pct_change"] = (donors_15_2nd_qtr["total_inside_bill"] - donors_15_2nd_qtr["total_outside_bill"]) / donors_15_2nd_qtr["total_outside_bill"].abs()
donors_15_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
7,U0000000074,[ABC Supply],"[HENDRICKS, DIANE, HENDRICKS, DIANE M MS]","5,000,000.00","-1,712,400.00",1.52,-0.52,"6,712,400.00",3.92
43,U0000003690,"[[24T Contribution], Uline Inc, None]","[UIHLEIN, RICHARD, UIHLEIN, RICHARD E, UIHLEIN...","3,196,100.00","2,143,200.00",0.60,0.40,"1,052,900.00",0.49
25,U0000003307,"[Bernard Marcus Family Foundation, None]","[MARCUS, BERNARD, MARCUS, BERNARD MR, MARCUS, ...","2,489,500.00","871,475.00",0.74,0.26,"1,618,025.00",1.86
59,U0000004245,[Susquehanna International Group],"[YASS, JANINE, YASS, JANINE MRS, YASS, JEFF, Y...","2,282,400.00","768,900.00",0.75,0.25,"1,513,500.00",1.97
5,U0000000066,[Elliott Management],"[SINGER, PAUL, SINGER, PAUL E MR, SINGER, PAUL...","1,681,590.00","4,904,992.00",0.26,0.74,"-3,223,402.00",-0.66


#### How much did they give?

In [116]:
print("2015 inside-bill 2nd quarter total: $" + str(donors_15_2nd_qtr["total_inside_bill"].sum()))
print("2015 outside-bill 2nd quarter total: $" + str(donors_15_2nd_qtr["total_outside_bill"].sum()))

2015 inside-bill 2nd quarter total: $33142895.0
2015 outside-bill 2nd quarter total: $74151346.0


### 2013

In [117]:
inside_bill_donors_13_2nd_qtr = inside_bill_donors_committees_13_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_13_2nd_qtr = outside_bill_donors_committees_13_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [118]:
donors_13_2nd_qtr = inside_bill_donors_13_2nd_qtr.merge(outside_bill_donors_13_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_13_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_13_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_13_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_13_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 5 columns):
match_id              158 non-null object
organizations         115 non-null object
contributors          115 non-null object
total_inside_bill     115 non-null float64
total_outside_bill    158 non-null float64
dtypes: float64(2), object(3)
memory usage: 7.4+ KB


In [119]:
donors_13_2nd_qtr["pct_inside_bill"] = donors_13_2nd_qtr["total_inside_bill"] / (donors_13_2nd_qtr["total_outside_bill"] + donors_13_2nd_qtr["total_inside_bill"])
donors_13_2nd_qtr["pct_outside_bill"] = donors_13_2nd_qtr["total_outside_bill"] / (donors_13_2nd_qtr["total_outside_bill"] + donors_13_2nd_qtr["total_inside_bill"])
donors_13_2nd_qtr["change"] = donors_13_2nd_qtr["total_inside_bill"] - donors_13_2nd_qtr["total_outside_bill"]
donors_13_2nd_qtr["pct_change"] = (donors_13_2nd_qtr["total_inside_bill"] - donors_13_2nd_qtr["total_outside_bill"]) / donors_13_2nd_qtr["total_outside_bill"].abs()
donors_13_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
52,U0000004245,[Susquehanna International Group],"[YASS, JANINE, YASS, JANINE MRS, YASS, JEFF, Y...","270,400.00","25,900.00",0.91,0.09,"244,500.00",9.44
37,U0000003682,[Renaissance Technologies],"[MERCER, DIANA L MS, MERCER, ROBERT, MERCER, R...","208,800.00","89,600.00",0.70,0.30,"119,200.00",1.33
46,U0000004017,"[Bluff Point Assoc, Rein Capitol LLC]","[MCINERNEY, PAULA, MCINERNEY, PAULA G MRS, MC ...","162,400.00","188,000.00",0.46,0.54,"-25,600.00",-0.14
32,U0000003614,[PVS Chemicals],"[NICHOLSON, ANN, NICHOLSON, ANN V, NICHOLSON, ...","123,300.00","78,800.00",0.61,0.39,"44,500.00",0.56
62,U0000004622,"[Ruffin Acquisitions LLC, Treasure Island]","[RUFFIN, PHILLIP, RUFFIN, PHILLIP MR]","116,070.00","9,600.00",0.92,0.08,"106,470.00",11.09


#### How much did they give?

In [120]:
print("2013 inside-bill 2nd quarter total: $" + str(donors_13_2nd_qtr["total_inside_bill"].sum()))
print("2013 outside-bill 2nd quarter total: $" + str(donors_13_2nd_qtr["total_outside_bill"].sum()))

2013 inside-bill 2nd quarter total: $3738394.0
2013 outside-bill 2nd quarter total: $11117095.0


### 2011

In [121]:
inside_bill_donors_11_2nd_qtr = inside_bill_donors_committees_11_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_11_2nd_qtr = outside_bill_donors_committees_11_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [122]:
donors_11_2nd_qtr = inside_bill_donors_11_2nd_qtr.merge(outside_bill_donors_11_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_11_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_11_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_11_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_11_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 5 columns):
match_id              172 non-null object
organizations         117 non-null object
contributors          117 non-null object
total_inside_bill     117 non-null float64
total_outside_bill    172 non-null float64
dtypes: float64(2), object(3)
memory usage: 8.1+ KB


In [123]:
donors_11_2nd_qtr["pct_inside_bill"] = donors_11_2nd_qtr["total_inside_bill"] / (donors_11_2nd_qtr["total_outside_bill"] + donors_11_2nd_qtr["total_inside_bill"])
donors_11_2nd_qtr["pct_outside_bill"] = donors_11_2nd_qtr["total_outside_bill"] / (donors_11_2nd_qtr["total_outside_bill"] + donors_11_2nd_qtr["total_inside_bill"])
donors_11_2nd_qtr["change"] = donors_11_2nd_qtr["total_inside_bill"] - donors_11_2nd_qtr["total_outside_bill"]
donors_11_2nd_qtr["pct_change"] = (donors_11_2nd_qtr["total_inside_bill"] - donors_11_2nd_qtr["total_outside_bill"]) / donors_11_2nd_qtr["total_outside_bill"].abs()
donors_11_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
23,U0000003585,"[Homemaker, Putnam Investments, None]","[REYNOLDS, LAURA C, REYNOLDS, ROBERT, REYNOLDS...","236,000.00","64,700.00",0.78,0.22,"171,300.00",2.65
32,U0000003819,[Caxton Alternative Management],"[KOVNER, BRUCE, KOVNER, BRUCE MR, KOVNER, SUZA...","180,000.00",0.00,1.00,0.00,"180,000.00",inf
13,U0000003273,[Harold Simmons Foundation],"[SIMMONS, ANNETTE]","138,500.00","5,646,100.00",0.02,0.98,"-5,507,600.00",-0.98
9,U0000002706,"[[24T Contribution], Ariel Corp]","[BUCHWALD-WRIGHT, KAREN MS, WRIGHT, KAREN, WRI...","137,100.00","34,300.00",0.80,0.20,"102,800.00",3.00
31,U0000003690,"[[24T Contribution], Uline Inc, None]","[UIHLEIN, RICHARD, UIHLEIN, RICHARD E, UIHLEIN...","135,000.00","446,000.00",0.23,0.77,"-311,000.00",-0.70


#### How much did they give?

In [124]:
print("2011 inside-bill 2nd quarter total: $" + str(donors_11_2nd_qtr["total_inside_bill"].sum()))
print("2011 outside-bill 2nd quarter total: $" + str(donors_11_2nd_qtr["total_outside_bill"].sum()))

2011 inside-bill 2nd quarter total: $2840199.0
2011 outside-bill 2nd quarter total: $18952905.0


### 2009

In [125]:
inside_bill_donors_09_2nd_qtr = inside_bill_donors_committees_09_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_09_2nd_qtr = outside_bill_donors_committees_09_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [126]:
donors_09_2nd_qtr = inside_bill_donors_09_2nd_qtr.merge(outside_bill_donors_09_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_09_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_09_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_09_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_09_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 0 to 132
Data columns (total 5 columns):
match_id              133 non-null object
organizations         77 non-null object
contributors          77 non-null object
total_inside_bill     77 non-null float64
total_outside_bill    133 non-null float64
dtypes: float64(2), object(3)
memory usage: 6.2+ KB


In [127]:
donors_09_2nd_qtr["pct_inside_bill"] = donors_09_2nd_qtr["total_inside_bill"] / (donors_09_2nd_qtr["total_outside_bill"] + donors_09_2nd_qtr["total_inside_bill"])
donors_09_2nd_qtr["pct_outside_bill"] = donors_09_2nd_qtr["total_outside_bill"] / (donors_09_2nd_qtr["total_outside_bill"] + donors_09_2nd_qtr["total_inside_bill"])
donors_09_2nd_qtr["change"] = donors_09_2nd_qtr["total_inside_bill"] - donors_09_2nd_qtr["total_outside_bill"]
donors_09_2nd_qtr["pct_change"] = (donors_09_2nd_qtr["total_inside_bill"] - donors_09_2nd_qtr["total_outside_bill"]) / donors_09_2nd_qtr["total_outside_bill"].abs()
donors_09_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
14,U0000003168,[TAMKO Building Products],"[HUMPHREYS, DAVID, HUMPHREYS, DAVID C, HUMPHRE...","74,600.00","62,800.00",0.54,0.46,"11,800.00",0.19
7,U0000000310,"[Adelson Drug Clinic, Interface Group, Las Veg...","[ADELSON, MIRIAM DR, ADELSON, MIRIAM DR DR, AD...","70,400.00","73,600.00",0.49,0.51,"-3,200.00",-0.04
23,U0000003463,"[[24T Contribution], Exoxemis Inc]","[STEPHENS, JACKSON T JR]","50,400.00","174,406.00",0.22,0.78,"-124,006.00",-0.71
18,U0000003273,[Harold Simmons Foundation],"[SIMMONS, ANNETTE]","47,200.00","91,500.00",0.34,0.66,"-44,300.00",-0.48
29,U0000003658,"[Franklin Resources, None]","[JOHNSON, ANN, JOHNSON, ANN L, JOHNSON, ANN L ...","45,400.00","62,200.00",0.42,0.58,"-16,800.00",-0.27


In [128]:
print("2009 inside-bill 2nd quarter total: $" + str(donors_09_2nd_qtr["total_inside_bill"].sum()))
print("2009 outside-bill 2nd quarter total: $" + str(donors_09_2nd_qtr["total_outside_bill"].sum()))

2009 inside-bill 2nd quarter total: $888939.0
2009 outside-bill 2nd quarter total: $3713898.0


## How did each contributor's giving pattern change in each cycle compared with 2017?

### 2015

In [129]:
donors_15_17_2nd_qtr = donors_17_2nd_qtr.merge(donors_15_2nd_qtr, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17_2nd_qtr.drop(["change_17", "pct_change_17", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17_2nd_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_15_17_2nd_qtr = donors_15_17_2nd_qtr[["match_id", "contributors", "organizations", "total_outside_bill_15", "total_inside_bill_15",
                 "pct_outside_bill_15", "pct_inside_bill_15", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_15_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219 entries, 0 to 218
Data columns (total 11 columns):
match_id                 219 non-null object
contributors             219 non-null object
organizations            219 non-null object
total_outside_bill_15    186 non-null float64
total_inside_bill_15     149 non-null float64
pct_outside_bill_15      149 non-null float64
pct_inside_bill_15       149 non-null float64
total_outside_bill_17    219 non-null float64
total_inside_bill_17     219 non-null float64
pct_outside_bill_17      219 non-null float64
pct_inside_bill_17       219 non-null float64
dtypes: float64(8), object(3)
memory usage: 20.5+ KB


In [130]:
donors_15_17_2nd_qtr["giving_change"] = np.where((donors_15_17_2nd_qtr["pct_inside_bill_17"] > donors_15_17_2nd_qtr["pct_inside_bill_15"]) | (donors_15_17_2nd_qtr["pct_outside_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17_2nd_qtr["pct_inside_bill_17"] < donors_15_17_2nd_qtr["pct_inside_bill_15"], "Decreased",
                                           np.where(donors_15_17_2nd_qtr["pct_inside_bill_17"] == donors_15_17_2nd_qtr["pct_inside_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17_2nd_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_15,total_inside_bill_15,pct_outside_bill_15,pct_inside_bill_15,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Ak-Chin Indian Community,"[AK-CHIN INDIAN COMMUNITY, AK-CHIN INDIAN COMM...",[Ak-Chin Indian Community],"15,200.00","110,000.00",0.12,0.88,"6,000.00","93,400.00",0.06,0.94,Increased


### 2013

In [131]:
donors_13_17_2nd_qtr = donors_17_2nd_qtr.merge(donors_13_2nd_qtr, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17_2nd_qtr.drop(["change_17", "pct_change_17", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17_2nd_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_13_17_2nd_qtr = donors_13_17_2nd_qtr[["match_id", "contributors", "organizations", "total_outside_bill_13", "total_inside_bill_13",
                 "pct_outside_bill_13", "pct_inside_bill_13", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_13_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219 entries, 0 to 218
Data columns (total 11 columns):
match_id                 219 non-null object
contributors             219 non-null object
organizations            219 non-null object
total_outside_bill_13    158 non-null float64
total_inside_bill_13     115 non-null float64
pct_outside_bill_13      115 non-null float64
pct_inside_bill_13       115 non-null float64
total_outside_bill_17    219 non-null float64
total_inside_bill_17     219 non-null float64
pct_outside_bill_17      219 non-null float64
pct_inside_bill_17       219 non-null float64
dtypes: float64(8), object(3)
memory usage: 20.5+ KB


In [132]:
donors_13_17_2nd_qtr["giving_change"] = np.where((donors_13_17_2nd_qtr["pct_inside_bill_17"] > donors_13_17_2nd_qtr["pct_inside_bill_13"]) | (donors_13_17_2nd_qtr["pct_outside_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17_2nd_qtr["pct_inside_bill_17"] < donors_13_17_2nd_qtr["pct_inside_bill_13"], "Decreased",
                                           np.where(donors_13_17_2nd_qtr["pct_inside_bill_17"] == donors_13_17_2nd_qtr["pct_inside_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17_2nd_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_13,total_inside_bill_13,pct_outside_bill_13,pct_inside_bill_13,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Ak-Chin Indian Community,"[AK-CHIN INDIAN COMMUNITY, AK-CHIN INDIAN COMM...",[Ak-Chin Indian Community],"62,800.00","49,600.00",0.56,0.44,"6,000.00","93,400.00",0.06,0.94,Increased


### 2011

In [133]:
donors_11_17_2nd_qtr = donors_17_2nd_qtr.merge(donors_11_2nd_qtr, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17_2nd_qtr.drop(["change_17", "pct_change_17", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17_2nd_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_11_17_2nd_qtr = donors_11_17_2nd_qtr[["match_id", "contributors", "organizations", "total_outside_bill_11", "total_inside_bill_11",
                 "pct_outside_bill_11", "pct_inside_bill_11", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_11_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219 entries, 0 to 218
Data columns (total 11 columns):
match_id                 219 non-null object
contributors             219 non-null object
organizations            219 non-null object
total_outside_bill_11    172 non-null float64
total_inside_bill_11     117 non-null float64
pct_outside_bill_11      117 non-null float64
pct_inside_bill_11       117 non-null float64
total_outside_bill_17    219 non-null float64
total_inside_bill_17     219 non-null float64
pct_outside_bill_17      219 non-null float64
pct_inside_bill_17       219 non-null float64
dtypes: float64(8), object(3)
memory usage: 20.5+ KB


In [134]:
donors_11_17_2nd_qtr["giving_change"] = np.where((donors_11_17_2nd_qtr["pct_inside_bill_17"] > donors_11_17_2nd_qtr["pct_inside_bill_11"]) | (donors_11_17_2nd_qtr["pct_outside_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17_2nd_qtr["pct_inside_bill_17"] < donors_11_17_2nd_qtr["pct_inside_bill_11"], "Decreased",
                                           np.where(donors_11_17_2nd_qtr["pct_inside_bill_17"] == donors_11_17_2nd_qtr["pct_inside_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17_2nd_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_11,total_inside_bill_11,pct_outside_bill_11,pct_inside_bill_11,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Ak-Chin Indian Community,"[AK-CHIN INDIAN COMMUNITY, AK-CHIN INDIAN COMM...",[Ak-Chin Indian Community],0.00,"1,000.00",0.00,1.00,"6,000.00","93,400.00",0.06,0.94,Decreased


### 2009

In [135]:
donors_09_17_2nd_qtr = donors_17_2nd_qtr.merge(donors_09_2nd_qtr, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17_2nd_qtr.drop(["change_17", "pct_change_17", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17_2nd_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_09_17_2nd_qtr = donors_09_17_2nd_qtr[["match_id", "contributors", "organizations", "total_outside_bill_09", "total_inside_bill_09",
                 "pct_outside_bill_09", "pct_inside_bill_09", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_09_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219 entries, 0 to 218
Data columns (total 11 columns):
match_id                 219 non-null object
contributors             219 non-null object
organizations            219 non-null object
total_outside_bill_09    133 non-null float64
total_inside_bill_09     77 non-null float64
pct_outside_bill_09      77 non-null float64
pct_inside_bill_09       77 non-null float64
total_outside_bill_17    219 non-null float64
total_inside_bill_17     219 non-null float64
pct_outside_bill_17      219 non-null float64
pct_inside_bill_17       219 non-null float64
dtypes: float64(8), object(3)
memory usage: 20.5+ KB


In [136]:
donors_09_17_2nd_qtr["giving_change"] = np.where((donors_09_17_2nd_qtr["pct_inside_bill_17"] > donors_09_17_2nd_qtr["pct_inside_bill_09"]) | (donors_09_17_2nd_qtr["pct_outside_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17_2nd_qtr["pct_inside_bill_17"] < donors_09_17_2nd_qtr["pct_inside_bill_09"], "Decreased",
                                           np.where(donors_09_17_2nd_qtr["pct_inside_bill_17"] == donors_09_17_2nd_qtr["pct_inside_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17_2nd_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_09,total_inside_bill_09,pct_outside_bill_09,pct_inside_bill_09,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Ak-Chin Indian Community,"[AK-CHIN INDIAN COMMUNITY, AK-CHIN INDIAN COMM...",[Ak-Chin Indian Community],nan,nan,nan,nan,"6,000.00","93,400.00",0.06,0.94,Increased


## What proportion of donors increased the share of their annual giving that fell within the two-month period in 2017 as compared with the same period in each of the previous years?

In [137]:
print(str(donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_15_17_2nd_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2015 and 2017.")
print(str(donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_13_17_2nd_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2013 and 2017.")
print(str(donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_11_17_2nd_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2011 and 2017.")
print(str(donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_09_17_2nd_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2009 and 2017.")

81.7351598173516% of donors increased the share of their annual giving that fell within the two-month period between 2015 and 2017.
84.47488584474885% of donors increased the share of their annual giving that fell within the two-month period between 2013 and 2017.
87.21461187214612% of donors increased the share of their annual giving that fell within the two-month period between 2011 and 2017.
91.78082191780823% of donors increased the share of their annual giving that fell within the two-month period between 2009 and 2017.


## And by how much did these individuals' giving increase?

In [138]:
print("These donors' giving within the two-month period increased by $" + str(donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_15"].sum()) + ", from $" + str(donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_15"].sum()) + " to $" + str(donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2015 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_13"].sum()) + ", from $" + str(donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_13"].sum()) + " to $" + str(donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2013 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_11"].sum()) + ", from $" + str(donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_11"].sum()) + " to $" + str(donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2011 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_09"].sum()) + ", from $" + str(donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_09"].sum()) + " to $" + str(donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2009 and 2017.")

These donors' giving within the two-month period increased by $24389803.0, from $9480863.0 to $33870666.0 between 2015 and 2017.
These donors' giving within the two-month period increased by $35387586.0, from $1442575.0 to $36830161.0 between 2013 and 2017.
These donors' giving within the two-month period increased by $34047447.0, from $1354150.0 to $35401597.0 between 2011 and 2017.
These donors' giving within the two-month period increased by $36385424.0, from $649739.0 to $37035163.0 between 2009 and 2017.


## How many donors in each year gave all of their money in the two-month period of each year?

In [139]:
print("In 2017, " + str(donors_17_2nd_qtr[donors_17_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2015, " + str(donors_15_2nd_qtr[donors_15_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2013, " + str(donors_13_2nd_qtr[donors_13_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2011, " + str(donors_11_2nd_qtr[donors_11_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2009, " + str(donors_09_2nd_qtr[donors_09_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")

In 2017, 40 donors gave all their money in the two-month period.
In 2015, 5 donors gave all their money in the two-month period.
In 2013, 8 donors gave all their money in the two-month period.
In 2011, 9 donors gave all their money in the two-month period.
In 2009, 6 donors gave all their money in the two-month period.


# What does this giving look like on a contribution-by-contribution level?

In [140]:
def run_contributions_query_2nd_qtr(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start_one, end_one, start_two, end_two):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_contributions_""" + suffix + """ AS
    SELECT fectransid,
    match_id,
    organization,
    contributor, date, sum(SUM),
    cmte_id,
    pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
                       WHERE date >= '""" + start_one + """'
                         AND date <= '""" + end_one + """'
                         or (date >= '""" + start_two + """'
                         and date <= '""" + end_two + """')
                         AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;
                            SELECT *
                            FROM """ + prefix + """_contributions_"""+ suffix +""";"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [141]:
inside_bill_contributions_committees_17_2nd_qtr = run_contributions_query_2nd_qtr("inside", "17_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2018", "2017-05-02", "2017-06-30", "2017-05-02", "2017-06-30")
outside_bill_contributions_committees_17_2nd_qtr = run_contributions_query_2nd_qtr("outside", "17_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2018", "2017-01-01", "2017-05-01", "2017-07-01", "2017-12-31")

In [142]:
inside_bill_contributions_committees_15_2nd_qtr = run_contributions_query_2nd_qtr("inside", "15_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2016", "2015-05-02", "2015-06-30", "2015-05-02", "2015-06-30")
outside_bill_contributions_committees_15_2nd_qtr = run_contributions_query_2nd_qtr("outside", "15_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2016", "2015-01-01", "2015-05-01", "2015-07-01", "2015-12-31")

In [143]:
inside_bill_contributions_committees_13_2nd_qtr = run_contributions_query_2nd_qtr("inside", "13_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2014", "2013-05-02", "2013-06-30", "2013-05-02", "2013-06-30")
outside_bill_contributions_committees_13_2nd_qtr = run_contributions_query_2nd_qtr("outside", "13_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2014", "2013-01-01", "2013-05-01", "2013-07-01", "2013-12-31")

In [144]:
inside_bill_contributions_committees_11_2nd_qtr = run_contributions_query_2nd_qtr("inside", "11_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2012", "2011-05-02", "2011-06-30", "2011-05-02", "2011-06-30")
outside_bill_contributions_committees_11_2nd_qtr = run_contributions_query_2nd_qtr("outside", "11_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2012", "2011-01-01", "2011-05-01", "2011-07-01", "2011-12-31")

In [145]:
inside_bill_contributions_committees_09_2nd_qtr = run_contributions_query_2nd_qtr("inside", "09_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2010", "2009-05-02", "2009-06-30", "2009-05-02", "2009-06-30")
outside_bill_contributions_committees_09_2nd_qtr = run_contributions_query_2nd_qtr("outside", "09_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2010", "2009-01-01", "2009-05-01", "2009-07-01", "2009-12-31")

## Export data to Excel.

In [146]:
writer = pd.ExcelWriter("data/megadonors.xlsx")
donors_15_17.to_excel(writer, "donors_15_17", startcol=0, index=False)
donors_13_17.to_excel(writer, "donors_13_17", startcol=0, index=False)
donors_11_17.to_excel(writer, "donors_11_17", startcol=0, index=False)
donors_09_17.to_excel(writer, "donors_09_17", startcol=0, index=False)
post_bill_donors_committees_17.to_excel(writer, "post_bill_donors_committees_17", startcol=0, index=False)
pre_bill_donors_committees_17.to_excel(writer, "pre_bill_donors_committees_17", startcol=0, index=False)
post_bill_donors_committees_15.to_excel(writer, "post_bill_donors_committees_15", startcol=0, index=False)
pre_bill_donors_committees_15.to_excel(writer, "pre_bill_donors_committees_15", startcol=0, index=False)
post_bill_donors_committees_13.to_excel(writer, "post_bill_donors_committees_13", startcol=0, index=False)
pre_bill_donors_committees_13.to_excel(writer, "pre_bill_donors_committees_13", startcol=0, index=False)
post_bill_donors_committees_11.to_excel(writer, "post_bill_donors_committees_11", startcol=0, index=False)
pre_bill_donors_committees_11.to_excel(writer, "pre_bill_donors_committees_11", startcol=0, index=False)
post_bill_donors_committees_09.to_excel(writer, "post_bill_donors_committees_09", startcol=0, index=False)
pre_bill_donors_committees_09.to_excel(writer, "pre_bill_donors_committees_09", startcol=0, index=False)
post_bill_contributions_17.to_excel(writer, "post_bill_contributions_17", startcol=0, index=False)
pre_bill_contributions_17.to_excel(writer, "pre_bill_contributions_17", startcol=0, index=False)
post_bill_contributions_15.to_excel(writer, "post_bill_contributions_15", startcol=0, index=False)
pre_bill_contributions_15.to_excel(writer, "pre_bill_contributions_15", startcol=0, index=False)
post_bill_contributions_13.to_excel(writer, "post_bill_contributions_13", startcol=0, index=False)
pre_bill_contributions_13.to_excel(writer, "pre_bill_contributions_13", startcol=0, index=False)
post_bill_contributions_11.to_excel(writer, "post_bill_contributions_11", startcol=0, index=False)
pre_bill_contributions_11.to_excel(writer, "pre_bill_contributions_11", startcol=0, index=False)
post_bill_contributions_09.to_excel(writer, "post_bill_contributions_09", startcol=0, index=False)
pre_bill_contributions_09.to_excel(writer, "pre_bill_contributions_09", startcol=0, index=False)
democratic_donors_17.to_excel(writer, "democratic_donors_17", startcol=0, index=False)
post_bill_democratic_donors_committees_17.to_excel(writer, "post_bill_dem_donors_cmtes_17", startcol=0, index=False)
pre_bill_democratic_donors_committees_17.to_excel(writer, "pre_bill_dem_donors_cmtes_17", startcol=0, index=False)
post_bill_democratic_contributions_17.to_excel(writer, "post_bill_dem_contributions_17", startcol=0, index=False)
pre_bill_democratic_contributions_17.to_excel(writer, "pre_bill_dem_contributions_17", startcol=0, index=False)
writer.save()